In [ ]:
---
title: "Housing"
format: html
bibliography: references.bib
---

In [2]:
#| output: false
#| echo: false

%load_ext pycodestyle_magic
%pycodestyle_on

1:1: E265 block comment should start with '# '
2:1: E265 block comment should start with '# '


The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


### Note:

Before running this notebook, please download [listings.csv.gz](http://data.insideairbnb.com/the-netherlands/north-holland/amsterdam/2023-09-03/data/listings.csv.gz) and place the extracted `listings.csv` in the `data` directory. 

# Assignment 3 : Housing

### Preliminary Data
#### The following infomation was found prior to the start of the assignment in order to use it.

tax hotels = 7% of turnover + 3 euros per room
tax airbnb = 10% = 3 euros
number of visitors = 30000

Library imports and DataFrame generation

In [112]:
import pandas as pd
import geopandas as gpd

csv_data = list()
listings = open("C:/Users/jjcor/Downloads/listings_full.csv",'r', encoding='latin1') #"./data/listings.csv"
for line in listings:
    data = line.split(',')
    csv_data.append(data)

listings.close()

df = pd.read_csv("C:/Users/jjcor/Downloads/listings_full.csv", encoding='latin1')


#### 1. What Amsterdam will receive from tourist tax if the event lasts a week and you will have 30.000 visitors?

#### Pseudocode

1. Discard listings with minimum stays of more than 7 nights
2. Sum the total ammount of people that fit the remaining airbnbs accomodate (#19812)
3. Calculate average price of accomodation remaining listings
4. for the remaining visitors (11188) they will stay in hotels --> average of 2.2 beds per room
5. Divide the number of rooms between the remaining tourists
6. Multiply number of available beds to average price
7. Sum airbnb + hotels taxes accordingly

In [ ]:
#Gathering information shortcuts:
 
#ok_listings = pd.DataFrame(df.columns)
#print(df.columns)
#print(df.columns[34])
#print(df.maximum_nights)

### Code:

In [37]:
#we discard listings with minimum stays of more than 7 nights
ok_listings = df[(df['minimum_nights'] < 8)].copy()

#we check that all numeric values in our DataFrame sum
test = ok_listings.sum(axis=0,numeric_only=True)

#we sum individually the column 'accommodates' so we can retrieve the number of people who can stay 
no_ppl = ok_listings.accommodates.sum()

print('Number of Airbnb listings with at most 7 days of minimal nights :', no_ppl)
#print(test) success

tourists = 30000

tourists_hotel = tourists-no_ppl

print('Number of people who needs to stay in hotel rooms',': ', tourists_hotel)
#print(ok_listings)

#Average cost of overnight accommodation in Amsterdam in the Netherlands from January 2019 to June 2023
##https://www.statista.com/statistics/614061/overnight-accommodation-costs-amsterdam-city/

hotel_price_per_night = 294

#https://www.cbs.nl/en-gb/figures/detail/84040ENG
#Hotels; capacity, type of accommodation, beds, star rating

hotel_rooms_ams = 41840
hotel_beds_ams = 90918

avg_beds_per_room = hotel_beds_ams/hotel_rooms_ams

#print(avg_beds_per_room)

#Number of hotel rooms used by tourists who don't fit in airbnbs
no_hotel_rooms_used = tourists_hotel/avg_beds_per_room

revenue_hotels = no_hotel_rooms_used*hotel_price_per_night

tax_hotels = revenue_hotels * 0.07 + no_hotel_rooms_used * 3

#getting rid of $ signs in column
new_price = df['price'].str.slice(1,-1)

#adding new_price as a column in our dataframe
ok_listings['new_price'] = new_price

ok_listings['new_price']=ok_listings['new_price'].str.replace(',','')

#converting new column values to float so we can math it up =P
ok_listings['new_price'] = ok_listings['new_price'].astype(float)

#calculating total revenue of airbnb by summing all location price per night and then times 7 (number of days in a week)
revenue_airbnb = ok_listings.new_price.sum()*7

tax_airbnb = float(revenue_airbnb)*.1 + float(len(new_price)*3)

print('Total tax revenue from Airbnb locations $', tax_airbnb)
print('Total tax revenue from hotel rooms $',tax_hotels)

total_revenue = tax_airbnb + tax_hotels

print('Total tax revenue gathered for the government', total_revenue)

Number of Airbnb listings with at most 7 days of minimal nights : 23407.0
Number of people who needs to stay in hotel rooms :  6593.0
Total tax revenue from Airbnb locations $ 1469366.9000000001
Total tax revenue from hotel rooms $ 71543.25226687785
Total tax revenue gathered for the government 1540910.152266878


### 2. Plot the amount of AirBnB locations per neighbourhood.

### Pseudocode
1. Filter from dataframe the locations of airbnb
2. Gather the different categories inside the column neighourhood
3. Sum each category inside neighbourhood

In [66]:
#It can't get mpre efficient than this
import plotly.express as px
#table = ok_listings.neighbourhood_cleansed.value_counts()
re_index = ok_listings['neighbourhood_cleansed'].value_counts().reset_index().rename(columns={"index": 'neighbourhood', 0 : 'count'})
print(re_index )

                    neighbourhood_cleansed  count
0                   De Baarsjes - Oud-West   1329
1                             Centrum-West    973
2                  De Pijp - Rivierenbuurt    898
3                             Centrum-Oost    766
4                                     Zuid    571
5                               Westerpark    561
6                                 Oud-Oost    475
7                            Bos en Lommer    399
8                                Oud-Noord    389
9   Oostelijk Havengebied - Indische Buurt    304
10                         Watergraafsmeer    249
11                              Noord-West    212
12                IJburg - Zeeburgereiland    173
13                             Slotervaart    156
14                              Noord-Oost    135
15                 Geuzenveld - Slotermeer    110
16                  Buitenveldert - Zuidas     96
17                  De Aker - Nieuw Sloten     67
18                         Bijlmer-Centrum     47


In [67]:
#graph with information
figure = px.bar(re_index, x='neighbourhood_cleansed', y='count', title =  'Number of airbnbs per neighbourhood in Amsterdam')
figure.show()

### 3: Which street in Amsterdam has the most AirBnB apartments?

### Pseudocode
1. Find the locations of airbnb through lat and long.
2. Add the street category to the DataFrame.
3. Gather the different categories inside the column Street.
4. Sum each category per street filter.

In [121]:
#pip install geopy

import certifi
import ssl
import geopy.geocoders
from geopy.geocoders import Nominatim, Photon

# Workaround to fix SSL certififcate expired error on some laptops
# Solution taken from https://stackoverflow.com/a/50665487
# Error message:
#   GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): 
#   Max retries exceeded with url: /reverse?lat=52.40164&lon=4.95106&format=json&addressdetails=1 
#   (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] 
#   certificate verify failed: certificate has expired (_ssl.c:1006)')))
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx

geolocator = Photon(user_agent='geoapiExercises')

print(ok_listings.columns)

#New dataframe with only 3 parameters
geolocs_airbnb = ok_listings[['latitude', 'longitude']]

'''
#here we set up a geodata frame based on the airbnb listings and do so with lat and long as geo location
gdf_listings = gpd.GeoDataFrame(ok_listings[["ï»¿id", "neighbourhood", "license"]], geometry=gpd.points_from_xy(ok_listings.longitude, ok_listings.latitude), crs = "WGS84")
print(gdf_airbnb.head())
'''

geolocs_airbnb['longitude']=geolocs_airbnb['longitude'].str.replace(',','')
geolocs_airbnb.longitude = (geolocs_airbnb.longitude.str[:1] + geolocs_airbnb.longitude.str[1:]).astype(float)

#print(geolocs_airbnb.longitude)

geolocs_airbnb['latitude']=geolocs_airbnb['latitude'].str.replace(',','')
geolocs_airbnb.latitude = (geolocs_airbnb.latitude.str[:2] + '.' + geolocs_airbnb.latitude.str[2:]).astype(float)

#print(type(geolocs_airbnb.latitude[5]))

#print(longitude)
#print(geolocs_airbnb)
#print(len(geolocs_airbnb))
for i in range(len(geolocs_airbnb)):
    latitude, longitude = geolocs_airbnb.iloc[i]['latitude'],geolocs_airbnb.iloc[i]['longitude']
    print(f'{latitude}, {longitude}')
    location = geolocator.reverse((latitude, longitude), timeout=None)
    try:
        street = location.raw['address']['road']
        geolocs_airbnb.loc[i, 'street'] = street
    except:
        pass

geolocs_airbnb.head()

#adding street as a column in our dataframe
geolocs_airbnb['street'] = street


Index(['ï»¿id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 

C:\Users\jjcor\AppData\Local\Temp\ipykernel_5424\340548867.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\jjcor\AppData\Local\Temp\ipykernel_5424\340548867.py:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



ValueError: could not convert string to float: '52..37'

In [105]:
#print(geolocs_airbnb.street)
print('Number of Airbnb listings per street:', geolocs_airbnb.street.value_counts())

AttributeError: 'DataFrame' object has no attribute 'street'

### 4. Try to cross reference the data from the AirBnB dataset with the BBGA. Can you figure out if all apartments of AirBnB are designated as housing? Which number of apartments are not rented out all the time but are also used as normal housing?

The two datasets have no categories with immediate similar classification, therefore the cross reference of information is not possible.

### 5. How many hotel rooms should be built if Amsterdam wants to accommodate the same number of tourists?

In [ ]:
#we sum individually the column 'accommodates' so we can retrieve the number of people who can stay 
no_ppl = ok_listings.accommodates.sum()

tourists = 30000

tourists_hotel = tourists-no_ppl

hotel_rooms_ams = 41840
hotel_beds_ams = 90918

avg_beds_per_room = hotel_beds_ams/hotel_rooms_ams

#print(avg_beds_per_room)

#Number of hotel rooms used by tourists who don't fit in airbnbs
no_hotel_rooms_used = tourists_hotel/avg_beds_per_room

print('Number of hotel rooms used by tourists who do not fit in airbnbs:', no_hotel_rooms_used)

NameError: name 'ok_listings' is not defined

### 6. How many different licenses are issued?

In [ ]:
#verifying license column
#print(ok_listings.columns[68])
licenses = (ok_listings.license.nunique())
license_values =(ok_listings.license.value_counts())
print(license_values)
exemptions = (ok_listings.license.value_counts()['Exempt'])

ulicenses = licenses-exemptions
print('Number of unique Airbnb licenses in Amsterdam (not counting the exempt values):',ulicenses)

license
Exempt                      600
0363 78AD 8875 790E 3C05     14
ABCD 1234 AB12 89EF A0F9      9
036341086EC3C2FF2493          7
0363 A250 F710 53C5 1273      6
                           ... 
0363 D239 E048 910F 216B      1
0363 CB7E 060D 07E0 A4DE      1
0363 45B2 6D92 AA8B 1326      1
0363 F6A5 F87A D89D 8FC1      1
0363 2F20 4F53 FB57 2D7F      1
Name: count, Length: 6972, dtype: int64
Number of unique Airbnb licenses in Amsterdam (not counting the exempt values): 6372


### Results analysis: 
We have more listings than licenses and several 'exempt' values. This is because the listings already have another type of license since they either work as hotels or hostels. https://www.airbnb.com/help/article/860